__test for step 6__  

In [2]:
import numpy as np
from clarkewright import *  
from vendorpickup import *
from itertools import permutations    

In [5]:
#초기 거리 값  
# full_mat is travel_time_matrix of from vendor to vendor 
f = [0,1.6,0.6,2,1.7,1.6,1.5],[1.6,0,1.9,3.5,0.7,1.3,3.1],[0.6,1.9,0,1.6,2.2,2.2,1.6],[2,3.5,1.6,0,3.7,3.6,1.4],[1.7,0.7,2.2,3.7,0,0.7,3.1],[1.6,1.3,2.2,3.6,0.7,0,2.7], [1.5,3.1,1.6,1.4,3.1,2.7,0]
full_mat = np.array(f)
print(full_mat)
print(full_mat.shape)   
###Input 인수들  
# number of vehicle = 1
D = [1000,200,300,400,50,600,700] #Weekly demand of item i
C = [10,60,30,400,4000,200,40] #Unit cost of item i
Cr = 0.01 #Inventory holding cost (% of unit cost/week)
Cp = [0,0,0,0,0,0,0] #Setup cost to pick up item i
VVC = 12 #Variable vehicle cost ($/hour of travel time)
P = [[1],[2],[3],[4],[5],[6,7]] #Set of all items to be picked up from vendor k

[[0.  1.6 0.6 2.  1.7 1.6 1.5]
 [1.6 0.  1.9 3.5 0.7 1.3 3.1]
 [0.6 1.9 0.  1.6 2.2 2.2 1.6]
 [2.  3.5 1.6 0.  3.7 3.6 1.4]
 [1.7 0.7 2.2 3.7 0.  0.7 3.1]
 [1.6 1.3 2.2 3.6 0.7 0.  2.7]
 [1.5 3.1 1.6 1.4 3.1 2.7 0. ]]
(7, 7)


In [8]:
#dt_min, dt_max 구하기
dt_min, dt_max = cal_dtmin_dtmax(len(full_mat), full_mat)
print(dt_min) 
print(dt_max)
# n_max, n_min값 구하기
n_max,n_min = cal_nmax_nmin(D, C, Cr, Cp, VVC, P, dt_min, dt_max)
print(n_max,n_min)  

[2.0, 3.2, 3.0, 1.4, 2.0, 3.0]
[3.2, 3.2, 4.0, 3.4, 3.2, 3.0]
[1.44, 1.25, 1.12, 6.9, 6.45, 4.53] [1.14, 1.25, 0.97, 4.43, 5.1, 4.53]


In [15]:
nk, Mon, Tue, Wed, Thu, Fri = create_schedule(0, P, n_max, n_min)
travel_cost, schedules = calculate_total_week_cost(Mon, Tue, Wed, Thu, Fri, full_mat)
print(travel_cost)
print(schedules)

35.6
[[[0, 4, 5, 6, 0]], [[0, 4, 5, 6, 0]], [[0, 1, 5, 6, 3, 2, 0]], [[0, 4, 5, 6, 0]], [[0, 4, 5, 6, 0]]]


__Functions needed__  

In [11]:
# 모든 경우의 수를 뽑기 위한 모듈 import
from itertools import permutations

# 가능한 모든 경로를 추출하는 함수 (방문 노드는 고정, 여러 개의 루트가 나왔을 때도 사용 가능)
def get_everycase(routes):
    everycase = []
    for i in range(len(routes)):
        a = routes[i][1:-1]
        b = list(permutations(a,len(a)))
        e, routes_cases,routes_everycase = [], [], []
        
        for j in range(len(b)):        
            e.append(list(b[j]))
            routes_cases.append([0] + e[j] + [0])
            
        if len(routes) == 1:
            routes_everycase = routes_cases
            return routes_everycase
            break
            
        if len(everycase) == 0:
            for k in range(len(routes_cases)):
                d = []
                d.append(routes_cases[k])
                everycase.append(d)
                
        elif len(routes) == 2: 
            for k in range(len(everycase)):
                for l in range(len(routes_cases)):
                    everycase[k].append(routes_cases[l])
                    d3 = deepcopy(everycase)
                    routes_everycase.append(d3[k])
                    everycase[k].pop()
        else:
            m = 2
            while(m < len(routes)):
                for k in range(len(everycase)):
                    for l in range(len(routes_cases)):
                        everycase[k].append(routes_cases[l])
                        d3 = deepcopy(everycase)
                        routes_everycase.append(d3[k])
                        everycase[k].pop()
                d4 = deepcopy(routes_everycase)
                everycase = d4 
                m += 1
                break
                
    return routes_everycase

In [12]:
#요일별 모든 exchange 경로(index값)과 half_mat_day를 받았을 때, 최적 경로와 비용 계산 
def calculate_bestcost(routes_everycase, half_mat_day):
    #route에 따른 cost 리턴하기?  
    
    #half matrix를 전체 matrix로 확장(인접하게)
    #각 노드들 간의 최단 거리 행렬 
    changed_half_mat = np.copy(half_mat_day)
    for i in range(len(changed_half_mat)):
        for j in range(len(changed_half_mat)):
            if changed_half_mat[i,j] == -1:
                changed_half_mat[i,j] = 0
    
    #print(changed_half_mat)
    
    total_mat = changed_half_mat + np.transpose(changed_half_mat) 
    #print(total_mat)
    #routes에 따라서 cost 계산
    total_cost = 0
    best_cost = 10000
    
    for route_idx, route in enumerate(routes_everycase):
        print(route)
        for i in range(len(route)-1):  
            total_cost += total_mat[route[i],route[i+1]]
        total_cost = round(total_cost,10)
        print(f'{routes_everycase[route_idx]}일 때 비용 {total_cost}')
        if best_cost > total_cost :
            best_cost = total_cost
            best_route = routes_everycase[route_idx]
        total_cost = 0
    print(f' 최적 경로는 : {best_route}. 이 때의 비용은 : {best_cost}')     
    return best_cost, best_route

In [25]:
# 각 요일에 대해서 변형한 다음에 best schedule 추출해 주는 함수 구현
#우선 for문으로 
#

print(schedules) #월요일부터 금요일까지 schedule
names = ["Mon","Tue","Wed","Thu","Fri"]
days = [Mon, Tue, Wed, Thu, Fri]

total_week_cost = 0
local_optimal_schedule = []  #각 요일별로 best schedule 

for day_idx, day_schedule in enumerate(schedules):
    #print(day)
    #print()
    routes_everycase = get_everycase(day_schedule)
    print(routes_everycase)
    half_mat_day = to_half_mat_by_day(full_mat, days[day_idx])
    #print(half_mat_day) 
    #print(routes_everycase)
    best_cost, best_route = calculate_bestcost(routes_everycase, half_mat_day)
    #print(best_route)  
    local_optimal_schedule.append(best_route)  


[[[0, 4, 5, 6, 0]], [[0, 4, 5, 6, 0]], [[0, 1, 5, 6, 3, 2, 0]], [[0, 4, 5, 6, 0]], [[0, 4, 5, 6, 0]]]
[[0, 4, 5, 6, 0], [0, 4, 6, 5, 0], [0, 5, 4, 6, 0], [0, 5, 6, 4, 0], [0, 6, 4, 5, 0], [0, 6, 5, 4, 0]]


IndexError: index 4 is out of bounds for axis 1 with size 4